<a href="https://colab.research.google.com/github/NiyazovIlia/PyTorch-1/blob/lesson-6/HW_PyTorch_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
max_words = 2000
max_len = 300
num_classes = 1

# Training
epochs = 5
batch_size = 512
print_batch_n = 100

In [3]:
data = pd.read_csv('train.csv')

# train, valid = train_test_split(data, test_size=0.3, random_state=42)

test = pd.read_csv('test.csv')

In [4]:
print(data.shape)
data.head()

(31962, 3)


,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [5]:
print(test.shape)
test.tail()

(17197, 2)


,id,tweet
17192,49155,thought factory: left-right polarisation! #tru...
17193,49156,feeling like a mermaid ð #hairflip #neverre...
17194,49157,#hillary #campaigned today in #ohio((omg)) &am...
17195,49158,"happy, at work conference: right mindset leads..."
17196,49159,"my song ""so glad"" free download! #shoegaze ..."


In [6]:
data['label'].value_counts()

0    29720
1     2242
Name: label, dtype: int64

In [7]:
data.loc[data['label'] == 1,:]

,id,label,tweet
13,14,1,@user #cnn calls #michigan middle school 'buil...
14,15,1,no comment! in #australia #opkillingbay #se...
17,18,1,retweet if you agree!
23,24,1,@user @user lumpy says i am a . prove it lumpy.
34,35,1,it's unbelievable that in the 21st century we'...
...,...,...,...
31934,31935,1,lady banned from kentucky mall. @user #jcpenn...
31946,31947,1,@user omfg i'm offended! i'm a mailbox and i'...
31947,31948,1,@user @user you don't have the balls to hashta...
31948,31949,1,"makes you ask yourself, who am i? then am i a..."


In [8]:
data.loc[data['label'] == 0,:]

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation
...,...,...,...
31956,31957,0,off fishing tomorrow @user carnt wait first ti...
31957,31958,0,ate @user isz that youuu?ðððððð...
31958,31959,0,to see nina turner on the airwaves trying to...
31959,31960,0,listening to sad songs on a monday morning otw...


Если судить по данным то где label = 0 это позитивные, где label = 1 это негативные

Займемся предобработкой

In [9]:
!pip install stop-words pymorphy2

In [10]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [11]:
sw = set(get_stop_words("en"))

In [12]:
puncts = set(punctuation)
puncts

{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 '{',
 '|',
 '}',
 '~'}

In [13]:
morpher = MorphAnalyzer()


def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in puncts)
    txt = txt.lower()
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [14]:
data['tweet'].iloc[:1].values

array([' @user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run'],
      dtype=object)

In [15]:
data['tweet'].iloc[:1].apply(preprocess_text).values

array(['user father dysfunctional selfish drags kids dysfunction run'],
      dtype=object)

In [16]:
from tqdm import tqdm 
tqdm.pandas()

data['tweet'] = data['tweet'].progress_apply(preprocess_text)
test['tweet'] = test['tweet'].progress_apply(preprocess_text)

100%|██████████| 17197/17197 [00:09<00:00, 1824.42it/s]


In [17]:
train_corpus = " ".join(data["tweet"])
train_corpus = train_corpus.lower()

In [18]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)
tokens[:5]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['user', 'father', 'dysfunctional', 'selfish', 'drags']

Отфильтруем данные

и соберём в корпус N наиболее частых токенов

In [19]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [20]:
from nltk.probability import FreqDist

dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]  # вычитание 1 для padding
len(tokens_filtered_top)

1999

In [21]:
tokens_filtered_top[:10]

['user', 'love', 'day', 'happy', 'amp', 'just', 'will', 'im', 'u', 'time']

In [22]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}
vocabulary

{'user': 1,
 'love': 2,
 'day': 3,
 'happy': 4,
 'amp': 5,
 'just': 6,
 'will': 7,
 'im': 8,
 'u': 9,
 'time': 10,
 'life': 11,
 'like': 12,
 'today': 13,
 'new': 14,
 'now': 15,
 'positive': 16,
 'thankful': 17,
 'get': 18,
 'people': 19,
 'good': 20,
 'bihday': 21,
 'cant': 22,
 'one': 23,
 'see': 24,
 'can': 25,
 'dont': 26,
 'fathers': 27,
 'smile': 28,
 'go': 29,
 'want': 30,
 'take': 31,
 'work': 32,
 'healthy': 33,
 'got': 34,
 'weekend': 35,
 'fun': 36,
 'make': 37,
 'summer': 38,
 'best': 39,
 'friday': 40,
 'us': 41,
 'need': 42,
 'way': 43,
 'great': 44,
 'bull': 45,
 'family': 46,
 'beautiful': 47,
 '2': 48,
 'going': 49,
 'friends': 50,
 'days': 51,
 'first': 52,
 'really': 53,
 'wait': 54,
 'music': 55,
 'morning': 56,
 'back': 57,
 'know': 58,
 'world': 59,
 'tomorrow': 60,
 'fathersday': 61,
 'never': 62,
 'sad': 63,
 'week': 64,
 'cute': 65,
 'orlando': 66,
 'think': 67,
 'feel': 68,
 'blog': 69,
 'happiness': 70,
 'well': 71,
 'model': 72,
 'right': 73,
 'much': 74,
 

In [23]:
import numpy as np


def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])

    padding = [0] * (maxlen-len(result))
    return result[-maxlen:] + padding

In [24]:
train, valid = train_test_split(data, test_size=0.3, random_state=42)

In [25]:
%%time
x_train = np.asarray([text_to_sequence(text, max_len) for text in train["tweet"]])
x_valid = np.asarray([text_to_sequence(text, max_len) for text in valid["tweet"]])

CPU times: user 7.86 s, sys: 150 ms, total: 8.01 s
Wall time: 9.41 s


In [26]:
x_train.shape

(22373, 300)

In [27]:
train['tweet'].iloc[0]

'summer timeð\x9f\x8c summeriscoming swimming picoftheday tattoos issho pulsera fluyendo â\x80¦'

In [28]:
x_train[0]

array([  38, 1280,  598,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [29]:
from torch.utils.data import DataLoader, Dataset


class DataWrapper(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = torch.from_numpy(data).long()
        self.target = torch.from_numpy(target).long()
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        
        if self.transform:
            x = self.transform(x)
            
        return x, y
    
    def __len__(self):
        return len(self.data)

In [30]:
train_dataset = DataWrapper(x_train, train['label'].values)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = DataWrapper(x_valid, valid['label'].values)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

In [31]:
for x, l in train_loader:
    print(x.shape)
    print(l.shape)
    print(l[0])
    break

torch.Size([512, 300])
torch.Size([512])
tensor(0)


In [32]:
test_embed = [64,128,256]
test_vocab = [1000,1500,2000]

for embed in test_embed:
  for vocab in test_vocab:
    class Net(nn.Module):
        def __init__(self, vocab_size=vocab, embedding_dim=embed, out_channel=embed, num_classes = 1):
            super().__init__()
            self.embedding = nn.Embedding(vocab_size, embedding_dim)
            self.conv_1 = nn.Conv1d(embedding_dim, out_channel, kernel_size=2)
            self.conv_2 = nn.Conv1d(embedding_dim, out_channel, kernel_size=3)
            self.pool = nn.MaxPool1d(2)
            self.relu = nn.ReLU()
            self.linear_1 = nn.Linear(out_channel, out_channel // 2)
            self.linear_2 = nn.Linear(out_channel // 2, num_classes)
            
        def forward(self, x):        
            output = self.embedding(x)     
            output = output.permute(0, 2, 1)
            output = self.conv_1(output)
            output = self.relu(output)
            output = self.pool(output)

            output = self.conv_2(output)
            output = self.relu(output)
            output = self.pool(output)
            output = torch.max(output, axis=2).values
            output = self.linear_1(output)
            output = self.relu(output)
            output = self.linear_2(output)
            output = F.sigmoid(output)
            return output

    model = Net(vocab_size=max_words)

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    device

    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.BCELoss()


    model = model.to(device)
    model.train()
    th = 0.5

    train_loss_history = []
    test_loss_history = []


    for epoch in range(epochs):  
        running_items, running_right = 0.0, 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            
            # обнуляем градиент
            optimizer.zero_grad()
            outputs = model(inputs)
            
            loss = criterion(outputs, labels.float().view(-1, 1))
            loss.backward()
            optimizer.step()

            # подсчет ошибки на обучении
            loss = loss.item()
            running_items += len(labels)
            # подсчет метрики на обучении
            pred_labels = torch.squeeze((outputs > th).int())
            running_right += (labels == pred_labels).sum()
            
            # выводим статистику о процессе обучения
            if i % 150 == 0:    # печатаем каждые 150 batches
                model.eval()
                
                print(f'Epoch [{epoch + 1}/{epochs}]. ' \
                      f'Step [{i + 1}/{len(train_loader)}]. ' \
                      f'Loss: {loss:.3f}. ' \
                      f'Acc: {running_right / running_items:.3f}', end='. ')
                running_loss, running_items, running_right = 0.0, 0.0, 0.0
                train_loss_history.append(loss)

                # выводим статистику на тестовых данных
                test_running_right, test_running_total, test_loss = 0.0, 0.0, 0.0
                for j, data in enumerate(val_loader):
                    test_labels = data[1].to(device)
                    test_outputs = model(data[0].to(device))
                    
                    # подсчет ошибки на тесте
                    test_loss = criterion(test_outputs, test_labels.float().view(-1, 1))
                    # подсчет метрики на тесте
                    test_running_total += len(data[1])
                    pred_test_labels = torch.squeeze((test_outputs > th).int())
                    test_running_right += (test_labels == pred_test_labels).sum()
                
                test_loss_history.append(test_loss.item())
                print(f'Test loss: {test_loss:.3f}. Test acc: {test_running_right / test_running_total:.3f}')
            
            model.train()
            
    print(f'Training with {embed} embed and {vocab} vocab_size is finished!')
print('Training is finished!')

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch [1/5]. Step [1/44]. Loss: 0.656. Acc: 0.932. Test loss: 0.351. Test acc: 0.929
Epoch [2/5]. Step [1/44]. Loss: 0.165. Acc: 0.938. Test loss: 0.258. Test acc: 0.929
Epoch [3/5]. Step [1/44]. Loss: 0.131. Acc: 0.957. Test loss: 0.030. Test acc: 0.938
Epoch [4/5]. Step [1/44]. Loss: 0.059. Acc: 0.979. Test loss: 0.154. Test acc: 0.948
Epoch [5/5]. Step [1/44]. Loss: 0.075. Acc: 0.973. Test loss: 0.005. Test acc: 0.943
Training with 64 embed and 1000 vocab_size is finished!
Epoch [1/5]. Step [1/44]. Loss: 0.692. Acc: 0.479. Test loss: 0.280. Test acc: 0.929
Epoch [2/5]. Step [1/44]. Loss: 0.172. Acc: 0.941. Test loss: 0.049. Test acc: 0.929
Epoch [3/5]. Step [1/44]. Loss: 0.185. Acc: 0.910. Test loss: 0.783. Test acc: 0.929
Epoch [4/5]. Step [1/44]. Loss: 0.096. Acc: 0.959. Test loss: 0.007. Test acc: 0.945
Epoch [5/5]. Step [1/44]. Loss: 0.079. Acc: 0.979. Test loss: 1.382. Test acc: 0.947
Training with 64 embed and 1500 vocab_size is finished!
Epoch [1/5]. Step [1/44]. Loss: 0.724.

Как видно из тестов, при embedding_dim 256 заметно переобучение в связи с чем модель работет плохо